<a href="https://colab.research.google.com/github/SyedaJannatul/NLP-Project--Sentiment-Analysis/blob/main/Sentiment_Analysis_imdb_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow as tf

  Created wheel for tf: filename=tf-1.0.0-cp37-none-any.whl size=1285 sha256=b6781ce7d62d5905224e75fe86af64a79d57bb9d1376059579f0da31e7877c2b
  Stored in directory: /root/.cache/pip/wheels/7c/51/42/b034446bd6c8252825a1a0e7799b6ff633de376967db5ed5a6
Successfully built tf


In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install -q tfds-nightly

     |████████████████████████████████| 3.8MB 20.7MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset otherwise not
)

In [ ]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <SplitInfo num_examples=25000, num_shards=1

In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer()

def build_vocabulary():
    vocabulary = set()
    for text, _ in ds_train:
        vocabulary.update(tokenizer.tokenize(text.numpy().lower()))
    return vocabulary


vocabulary = build_vocabulary()

encoder = tfds.deprecated.text.TokenTextEncoder(list(vocabulary), oov_token="<UNK>", lowercase=True, tokenizer=tokenizer)

def my_enc(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label


def encode_map_fn(text, label):
    encoded_text, label = tf.py_function(my_enc, inp=[text, label], Tout=(tf.int64, tf.int64))# py_func doesn't set the shape of the returned tensors.
    #encoded_text.set_shape([None])# `tf.data.Datasets` work best if all components have a shape set so set the shapes manually.
    #label.set_shape([])
    return encoded_text, label


In [ ]:
# num_parallel_calls=AUTOTUNE; ->  it specifies How many parallel calls it's going to do,it is hyperparameter tensorflow has its own but we can set our own value.
# cache() -> this cache is essentially after the time its loaded the data,its going keep track of some of them in memory.so that its going to be faster for the next time.

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(encode_map_fn, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(1000)
ds_train = ds_train.padded_batch(32, padded_shapes=([None], ()))


ds_test = ds_test.map(encode_map_fn, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.padded_batch(32, padded_shapes=([None], ()))


In [ ]:
model = keras.Sequential(
    [
        layers.Masking(mask_value=0),
        layers.Embedding(input_dim=len(vocabulary) + 2, output_dim=32),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu"),
        layers.Dense(1),
    ]
)


In [ ]:
##Training  network configuration
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(3e-4, clipnorm=1),
    metrics=["accuracy"],
)


In [ ]:
#More of the concrete training of that network
model.fit(ds_train, epochs=10, verbose=2)
model.evaluate(ds_test, verbose=2)

Epoch 1/10


InvalidArgumentError: ignored